## Here is the summary of what are tried, what worked for me and what not
My baseline score is `86.23`. I believed it was due to `over-fitting`. And as the features are unknown, so it was difficult to make an interpretation for `feature engineering`. So i followed several approach to do that
1. Handle NULL value, with two ways
    - use tree-based method
    - created an indicator variable to represent `NULL` and fill it with some number in original feature
    - used MICE(didn't help)
2. Target encoding(didn't helped)(due to target leakage, tried to handle it carefully, but failed)
3. Removed highly correlated feature
4. Used `p-test` to remove irrelavant feature(It again can create some problem, as OLS is linear model)
    - I carefully selected `p-value`, one for linear model
    - other for further feature processing
5. SVD/PCA/NMF feature on original and selected dataset.
    - use NMF, we get 2 advantage
        1. it create new feature(as we need absolute value)
        2. feature space tramnformation
6. feature interaction, (i created a list of `20000` complex feature)
    - Select a subset of 300-400 feature, prepare dataset
    - build model to evaluate those feature
        1. If score < baseline: remove them
        2. Else: calculate `oof` prediction
7. created an ensemble
    - catboost/xgboost/logistic-regression/passive-aggressive/ridge
    
    
Even through all this, i could break my baseline.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import random
import warnings
warnings.filterwarnings("ignore")

import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

sns.set(context='notebook', style='whitegrid', palette='deep', 
        font='sans-serif', font_scale=2, color_codes=True, rc=None)

import os, gc
os.listdir('DataSet/')

['Train.csv', 'good_dummy_feature.csv', 'Sample_submission.csv', 'Test.csv']

In [2]:
train = pd.read_csv("DataSet/Train.csv")
test  = pd.read_csv("DataSet/Test.csv")

train.shape, test.shape

((17521, 2395), (20442, 2394))

In [3]:
train.drop(['Col747','Col836'], axis=1, inplace=True)
test.drop(['Col747','Col836'], axis=1, inplace=True)

for col in ['Col754','Col843','Col733', 'Col763', 'Col822', 'Col852', 'Col742', 'Col831']:
    test[col] = test[col].replace('-', np.nan).astype('float')
    train[col] = train[col].replace('-', np.nan).astype('float')

## handling null values

In [4]:
train = train.drop_duplicates()
nulls = train.isnull().sum(1).sort_values(ascending=False)

danger_idx = nulls[nulls >= 664].index
danger_idx = list(danger_idx)
len(danger_idx)

1094

In [5]:
print(train.shape, end=" ==> ")
train = train.drop(danger_idx, axis=0)
print(train.shape)

(17521, 2393) ==> (16427, 2393)


## grouping columns and remove some very bad feature(find using EDA)

In [6]:
import random, math

def grouping_cols(df, cat_percentage = 0.01, checking_itr = 20):
    cc, nc = [], []
    max_ = 0
    amount = int(df.shape[0]*cat_percentage)
    print(amount, "/", df.shape[0]," Used to differentiate num feature from cat feature")
    for col in df.columns:
        uni = df[col].unique().shape[0]
#         print(uni)
        max_ = max(max_, uni)
        if(uni <= amount):
            cc.append(col)
        else:
            nc.append(col)
        
    print("-----Filtered result after Ist stage-----")
    print("total cat cols: {}, total num cols: {}\n".format(len(cc), len(nc)))
    
    true_cat = []
    true_num = []
    for col in nc+cc:
        num = False
        if(df[col].dtype == 'object'):
            true_cat.append(col)
            continue
        for i in range(checking_itr):
            sample = np.random.choice(df[col].unique())
            if math.isnan(sample): continue
            if int(sample) != sample:
                num = True
        if num is True:
            true_num.append(col)
        else:
            true_cat.append(col)

    print("-----Filtered result after 2nd stage-----")
    print("total cat cols: {}, total num cols: {}".format(len(true_cat), len(true_num)))
    return cc, nc

cat_cols, num_cols = grouping_cols(train)


collect = ['Col190','Col191','Col192','Col204','Col230','Col242','Col252','Col253','Col912','Col347','Col384',
           'Col387','Col388','Col389','Col401','Col427','Col439','Col449','Col450','Col544','Col579','Col580',
           'Col581','Col582','Col583','Col584','Col585','Col586','Col587','Col597','Col603','Col623','Col635',
           'Col702','Col711','Col724','Col742','Col791','Col799','Col800','Col813','Col831']
for col in collect:
    if col in cat_cols:
        cat_cols.remove(col)
    if col not in num_cols:
        num_cols.append(col)
print(len(cat_cols), len(num_cols))

164 / 16427  Used to differentiate num feature from cat feature
-----Filtered result after Ist stage-----
total cat cols: 1648, total num cols: 745

-----Filtered result after 2nd stage-----
total cat cols: 1639, total num cols: 754
1611 782


In [7]:
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.metrics import classification_report as cr
from catboost import Pool, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb


In [8]:
def train_cat_model_orig(X_train, y_train, X_valid, y_valid, features, param, X_test, 
                    num_round):
    """
    Args:
        X_train, X_valid: training and valid data
        y_train, y_valid: training and valid target
        X_test: test-data
        features: training features
    Return:
        oof-pred, test_preds, model, model_imp
    """
    param['iterations'] = num_round
    
    _train = Pool(X_train[features], label=y_train)#, cat_features=cate_features_index)
    _valid = Pool(X_valid[features], label=y_valid)#, cat_features=cate_features_index)

    watchlist = [_train, _valid]
    clf = CatBoostClassifier(**param)
    clf.fit(_train, 
            eval_set=watchlist, 
            verbose=200,
            use_best_model=True)
        
    oof  = clf.predict_proba(X_valid[features])[:,1]
    test_pred  = clf.predict_proba(X_test[features])[:,1]

    cat_imp = pd.DataFrame(data=[clf.feature_names_, 
                                 list(clf.feature_importances_)]).T
    cat_imp.columns = ['feature','imp']
    
    return oof, test_pred, clf, cat_imp


def run_cv_cat(train_df, target, test_df, depth):

    params = {
        'loss_function'         : "Logloss", 
        'eval_metric'           : "AUC",
        'random_strength'       : 3.8,#1.5,
        'border_count'          : 128,
        'scale_pos_weight'      : 3.507,
        'depth'                 : depth, 
        'early_stopping_rounds' : 50,
        'random_seed'           : 1337,
        'task_type'             : 'CPU', 
#         'subsample'             = 0.7, 
        'iterations'            : 10000, 
        'learning_rate'         : 0.05,#0.09,
        'thread_count'          : 4,
        'l2_leaf_reg'           : 3.076,
    }
#     {'depth': 7.0,
#      'l2_leaf_reg': 3.0753960899722625,
#      'learning_rate': 0.063,
#      'random_strength': 3.8000000000000003}

    ##########################
    n_splits = 4
    random_seed = 1234
    feature_imp = pd.DataFrame()
    
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    oof_cat = np.zeros(len(train_df))
    predictions = np.zeros((len(test_df),n_splits))
    clfs = []
    f1s = []
##########################
    for fold_, (train_index, valid_index) in enumerate(folds.split(train_df, target)):
        print(train_index.shape, valid_index.shape)
        print("Fold {}".format(fold_))
    
        y_train, y_valid = target.iloc[train_index], target.iloc[valid_index]
        X_train, X_valid = train_df.iloc[train_index,:], train_df.iloc[valid_index,:]
        features = X_train.columns
        
        num_rounds = 10000
        oof, test_pred, clf, cat_imp = train_cat_model_orig(X_train, y_train, 
                                                       X_valid, y_valid, 
                                                       features, params, 
                                                       test_df, num_rounds)
    
        oof_cat[valid_index] = oof
        predictions[:,fold_] = test_pred
        
        cat_imp['fold'] = fold_
        feature_imp = pd.concat([feature_imp, cat_imp], axis=0)
        clfs.append(clf)
        
        score = roc_auc_score(y_valid, oof)
        print( "  auc = ", score )
        y_bin = [1. if y_cont > 0.5 else 0. for y_cont in oof]
        score = f1_score(y_valid, y_bin, average='weighted')
        print( "  f1 = ", score )
        print("="*60)
        gc.collect()
        f1s.append(score)
        
    feature_imp.imp = feature_imp.imp.astype('float')
    feature_imp = feature_imp.groupby(['feature'])['imp'].mean()
    feature_imp = pd.DataFrame(data=[feature_imp.index, feature_imp.values]).T
    feature_imp.columns=['feature','imp']
    feature_imp = feature_imp.sort_values(by='imp')

    return clfs, feature_imp, oof_cat, predictions, np.mean(f1s)


In [9]:
def train_xgb_model(X_train, y_train, X_valid, y_valid, depth=4):
    
    param = {
        'eval_metric'     : 'auc',
        'seed'            : 1337,
        'eta'             : 0.05,
        'subsample'       : 0.7,
        'colsample_bytree': 0.5,
        'silent'          : 1,
        'nthread'         : 4,
        'Scale_pos_weight': 100,#3.607,
        'objective'       : 'binary:logistic',
        'max_depth'       : depth,
        'alpha'           : 0.09
    }
    features = X_train.columns
    
    _train = xgb.DMatrix(X_train[features], label=y_train, feature_names=list(features))
    _valid = xgb.DMatrix(X_valid[features], label=y_valid,feature_names=list(features))
    
    watchlist = [(_valid, 'valid')]
    clf = xgb.train(dtrain=_train, 
                    num_boost_round=10000, 
                    evals=watchlist,
                    early_stopping_rounds=25, 
                    verbose_eval=200, 
                    params=param)
    
    valid_frame = xgb.DMatrix(X_valid[features],feature_names=list(features))
    oof  = clf.predict(valid_frame, ntree_limit=clf.best_ntree_limit)
    
    y_bin = [1. if y_cont > 0.5 else 0. for y_cont in oof]
    score = f1_score(y_valid, y_bin, average='weighted')
    print( "  f1 = ", score )
    print("="*60)

    xgb_imp = pd.DataFrame(data=[list(clf.get_fscore().keys()), 
                                 list(clf.get_fscore().values())]).T
    xgb_imp.columns = ['feature','imp']
    xgb_imp.imp = xgb_imp.imp.astype('float')
    xgb_imp = xgb_imp.sort_values(by='imp',ascending=False)
    
    return score, xgb_imp


In [10]:
def train_cat_model(X_train, y_train, X_valid, y_valid):
    param = {
        'loss_function'         : "Logloss", 
        'eval_metric'           : "AUC",
        'random_strength'       : 3.8,#1.5,
        'border_count'          : 128,
        'scale_pos_weight'      : 3.507,
        'depth'                 : 4, 
        'early_stopping_rounds' : 50,
        'random_seed'           : 1337,
        'task_type'             : 'CPU', 
#         'subsample'             = 0.7, 
        'iterations'            : 10000, 
        'learning_rate'         : 0.05,#0.09,
        'thread_count'          : 4,
        'l2_leaf_reg'           : 3.076,
    }
    param['iterations'] = 10000
    features = X_train.columns
    
    _train = Pool(X_train[features], label=y_train)#, cat_features=cate_features_index)
    _valid = Pool(X_valid[features], label=y_valid)#, cat_features=cate_features_index)

    watchlist = [_train, _valid]
    clf = CatBoostClassifier(**param)
    clf.fit(_train, 
            eval_set=watchlist, 
            verbose=0,
            use_best_model=True)
        
    oof  = clf.predict_proba(X_valid[features])[:,1]
    y_bin = [1. if y_cont > 0.5 else 0. for y_cont in oof]
    score = f1_score(y_valid, y_bin, average='weighted')
    print( "  f1 = ", score )
    print("="*60)
    
    cat_imp = pd.DataFrame(data=[clf.feature_names_, list(clf.feature_importances_)]).T
    cat_imp.columns = ['feature','imp']
    cat_imp = cat_imp.sort_values(by='imp', ascending=False)

    return score, cat_imp


In [11]:
# train = train.astype('category')
target = train['Col2']
tr_ids = train['Col1']
ts_ids = test['Col1']
train.drop(['Col1','Col2'], axis=1, inplace=True)
test.drop('Col1', axis=1, inplace=True)

## run statsmodel

In [12]:

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
np.random.seed(123)


corr = train.corr()

columns = np.full((corr.shape[0],), True, dtype=bool)
for i in range(corr.shape[0]):
    for j in range(i+1, corr.shape[0]):
        if corr.iloc[i,j] >= 0.9:
            if columns[j]:
                columns[j] = False

selected_columns = train.columns[columns]
print(len(selected_columns))


train1 = train[selected_columns]
test1 = test[selected_columns]
print(train1.shape, test1.shape)


501
(16427, 501) (20442, 501)


In [13]:
cat_cols, num_cols = grouping_cols(train1)
len(cat_cols), len(num_cols)

164 / 16427  Used to differentiate num feature from cat feature
-----Filtered result after Ist stage-----
total cat cols: 267, total num cols: 234

-----Filtered result after 2nd stage-----
total cat cols: 262, total num cols: 239


(267, 234)

In [14]:
train2 = train1.copy()
test2  = test1.copy()


In [15]:
new_num_cols = []
for col in num_cols:
    try:
        col_name = 'NULL_'+col
        
        min_tr = train1[col].min()
        var_tr = train1[col].var()
        
        min_ts = test1[col].min()
        var_ts = test1[col].var()
        
        train1[col] = train1[col].fillna(min_tr - var_tr/10)
        train1[col_name] = np.where(train[col].isnull(), 1, 0)
        
        test1[col] = test1[col].fillna(min_ts - var_ts/10)
        test1[col_name] = np.where(test[col].isnull(), 1, 0)
        
        # if get passed through all steps
        new_num_cols.append(col_name)
        
    except:
        print(col)

In [16]:
train1.shape

(16427, 735)

In [17]:
train1.fillna(-10, inplace=True)
test1.fillna(-10, inplace=True)


In [21]:
# selected_columns = selected_columns[1:].values
import statsmodels.formula.api as sm
def backwardElimination(x, Y, sl, columns):
    numVars = x.shape[1]
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues)#.astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    print(j, end=" ")
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    
    print(regressor_OLS.summary())
    return x, columns

SL = 0.05
data_modeled, selected_columns = backwardElimination(train1.values, target, SL, list(set(list(selected_columns)+new_num_cols)))
data_modeled = pd.DataFrame(data=data_modeled, columns=["col_"+str(i) for i in range(data_modeled.shape[1])])
data_modeled.shape

                            OLS Regression Results                            
Dep. Variable:                   Col2   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.106
Method:                 Least Squares   F-statistic:                     4.521
Date:                Mon, 30 Sep 2019   Prob (F-statistic):          2.60e-222
Time:                        02:15:48   Log-Likelihood:                -2293.7
No. Observations:               16427   AIC:                             5701.
Df Residuals:                   15870   BIC:                             9994.
Df Model:                         556                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -4.598e-09      1e-08     -0.458      0.6

(16427, 735)

In [22]:
print(train1[selected_columns].shape)

from sklearn.model_selection import train_test_split

X_tr, X_ts, y_tr, y_ts = train_test_split(train1[list(selected_columns)], target, stratify=target, test_size=0.3, random_state=1234)
print(X_tr.shape, X_ts.shape, y_tr.shape, y_ts.shape)

cat1 = train_xgb_model(X_tr, y_tr, X_ts, y_ts)

(16427, 735)
(11498, 735) (4929, 735) (11498,) (4929,)
[0]	valid-auc:0.625511
Will train until valid-auc hasn't improved in 25 rounds.
Stopping. Best iteration:
[63]	valid-auc:0.729896

  f1 =  0.8575995624552945


In [23]:
cat1 = run_cv_cat(train1[list(selected_columns)], target, test1[list(selected_columns)], depth=4)

(12320,) (4107,)
Fold 0
0:	test: 0.6167517	test1: 0.6198917	best: 0.6198917 (0)	total: 122ms	remaining: 20m 16s
200:	test: 0.7753188	test1: 0.7375813	best: 0.7376924 (188)	total: 9.87s	remaining: 8m 1s
400:	test: 0.8204815	test1: 0.7454502	best: 0.7455514 (397)	total: 20.2s	remaining: 8m 4s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.746850541
bestIteration = 455

Shrink model to first 456 iterations.
  auc =  0.7468505410312322
  f1 =  0.8756834573197397
(12320,) (4107,)
Fold 1
0:	test: 0.6310043	test1: 0.6456629	best: 0.6456629 (0)	total: 46.7ms	remaining: 7m 46s
200:	test: 0.7754024	test1: 0.7397367	best: 0.7397367 (200)	total: 9.47s	remaining: 7m 41s
400:	test: 0.8124222	test1: 0.7495068	best: 0.7496126 (396)	total: 18.7s	remaining: 7m 26s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.7499077675
bestIteration = 439

Shrink model to first 440 iterations.
  auc =  0.7499077674973983
  f1 =  0.8676953714108437
(12320,) (4107,)
Fold 2
0:	te

In [24]:
cat1[2]

array([0.35453718, 0.10610112, 0.10075505, ..., 0.14218191, 0.09735206,
       0.32669951])

In [25]:
ybins = [1.0 if y > 0.5 else 0 for y in cat1[2]]

from sklearn.metrics import classification_report as cr
print(cr(target, ybins))

ybins = [1.0 if y > 0.5 else 0 for y in np.mean(cat1[3], axis=1)]
sub_cat2 = pd.DataFrame(data=pd.concat([ts_ids.reset_index(drop=True), pd.DataFrame(ybins)], axis=1))
sub_cat2.columns = ['Col1','Col2']
sub_cat2.to_csv("predictions/ols_cat.csv", index=None)
sub_cat2.head(10)

              precision    recall  f1-score   support

           0       0.92      0.94      0.93     14791
           1       0.36      0.28      0.32      1636

   micro avg       0.88      0.88      0.88     16427
   macro avg       0.64      0.61      0.62     16427
weighted avg       0.87      0.88      0.87     16427



Col1  Col2
0  RIGD58ZWD   0.0
1  RIH660YDS   0.0
2  RIH660Q96   0.0
3  RIYDO15W1   0.0
4  RIYBGC1ZD   0.0
5  RIYBQ5SWR   0.0
6  RIYBQ5SD5   0.0
7   TI5HU5XY   0.0
8   TIYHGOSE   0.0
9   T90K61QJ   0.0

In [ ]:
np.sum(ybins)